In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 21.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL = "FILL"

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-28 19:51:21--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.04s   

2025-11-28 19:51:21 (5.38 MB/s) - ‘smsspamcollection.zip’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-28 19:51:21--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.03s   

2025-11-28 19:51:22 (6.64 MB/s) - ‘smsspamcollection.zip.1’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = df.text.values
labels = df.label.values

In [ ]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.do_lower_case = True

tokenizer.pad_token = "<|pad|>"
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'right'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

URGENT! We are trying to contact you. Last weekends draw shows that you have won a £900 prize GUARANTEED. Call 09061701939. Claim code S89. Valid 12hrs only

╒═════════════╤═════════════╕
│ Tokens      │   Token IDs │
╞═════════════╪═════════════╡
│ urg         │        3686 │
├─────────────┼─────────────┤
│ ent         │         298 │
├─────────────┼─────────────┤
│ !           │           0 │
├─────────────┼─────────────┤
│ Ġwe         │         356 │
├─────────────┼─────────────┤
│ Ġare        │         389 │
├─────────────┼─────────────┤
│ Ġtrying     │        2111 │
├─────────────┼─────────────┤
│ Ġto         │         284 │
├─────────────┼─────────────┤
│ Ġcontact    │        2800 │
├─────────────┼─────────────┤
│ Ġyou        │         345 │
├─────────────┼─────────────┤
│ .           │          13 │
├─────────────┼─────────────┤
│ Ġlast       │         938 │
├─────────────┼─────────────┤
│ Ġweekends   │       21511 │
├─────────────┼─────────────┤
│ Ġdraw       │        3197 │
├─

In [ ]:
print(text)
print(labels)
print(labels.size)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n'
 'Ok lar... Joking wif u oni...\n'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n"
 ... 'Pity, * was in mood for that. So...any other suggestions?\n'
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free\n"
 'Rofl. Its true to its name\n']
[0 0 1 ... 0 0 0]
5574


In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  encoding = tokenizer.encode_plus(
      input_text,
      add_special_tokens=True,
      max_length=32,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
    )
  return encoding

for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# [Batch]
labels = torch.tensor(labels)

In [ ]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['she', 'Ġjust', 'Ġbroke', 'Ġdown', 'Ġa', 'Ġlist', 'Ġof', 'Ġreasons', 'Ġwhy', 'Ġnobody', "'s", 'Ġin', 'Ġtown', 'Ġand', 'Ġi', 'Ġcan', "'t", 'Ġtell', 'Ġif', 'Ġshe', "'s", 'Ġbeing', 'Ġsarcastic', 'Ġor', 'Ġjust', 'Ġf', 'ag', 'gy', 'Ċ', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ she           │        7091 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġjust         │         655 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġbroke        │        6265 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġdown         │         866 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġa            │         257 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġlist         │        1351 │                1 │
├──

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(
    range(len(token_id)),
    test_size = val_ratio,
    stratify=labels,
    random_state = 69
)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(
    token_id[train_idx],
    attention_masks[train_idx],
    labels[train_idx]
)

val_set = TensorDataset(
    token_id[val_idx],
    attention_masks[val_idx],
    labels[val_idx]
)

# Prepare DataLoader
train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True
)

validation_dataloader = DataLoader(
    val_set,
    batch_size = batch_size
)

print(f"{len(train_set)} {len(val_set)}")

4459 1115


### Load specific versions of the model

In [ ]:
# Load the BertForSequenceClassification model ?
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('openai-community/gpt2',
                                    output_attentions=False,
                                    output_hidden_states=False)

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [ ]:
# If on GPU, do as below
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [ ]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = False

total_parameters = 0
for name, param in model.named_parameters():
    print(f"Layer: {name}, Parameters: {param.numel()}")
    total_parameters += param.numel()

# Turn off gradients using the above
if fine_tune:
  for param in model.parameters():
    param.requires_grad = True
else:
  for name, param in model.named_parameters():
    if 'score' not in name:
      param.requires_grad = False
    else:
      param.requires_grad = True

assert(total_parameters == 124441344)

Layer: transformer.wte.weight, Parameters: 38597376
Layer: transformer.wpe.weight, Parameters: 786432
Layer: transformer.h.0.ln_1.weight, Parameters: 768
Layer: transformer.h.0.ln_1.bias, Parameters: 768
Layer: transformer.h.0.attn.c_attn.weight, Parameters: 1769472
Layer: transformer.h.0.attn.c_attn.bias, Parameters: 2304
Layer: transformer.h.0.attn.c_proj.weight, Parameters: 589824
Layer: transformer.h.0.attn.c_proj.bias, Parameters: 768
Layer: transformer.h.0.ln_2.weight, Parameters: 768
Layer: transformer.h.0.ln_2.bias, Parameters: 768
Layer: transformer.h.0.mlp.c_fc.weight, Parameters: 2359296
Layer: transformer.h.0.mlp.c_fc.bias, Parameters: 3072
Layer: transformer.h.0.mlp.c_proj.weight, Parameters: 2359296
Layer: transformer.h.0.mlp.c_proj.bias, Parameters: 768
Layer: transformer.h.1.ln_1.weight, Parameters: 768
Layer: transformer.h.1.ln_1.bias, Parameters: 768
Layer: transformer.h.1.attn.c_attn.weight, Parameters: 1769472
Layer: transformer.h.1.attn.c_attn.bias, Parameters: 230

### Train the model

In [ ]:
device

'cuda'

In [ ]:
import torchmetrics
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = torchmetrics.Accuracy(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
precision = torchmetrics.Precision(task='binary').to(device)
auroc = torchmetrics.AUROC(task='binary').to(device)

In [ ]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        # Unpack the batch
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        loss = train_output.loss
        loss.backward()

        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        # Unpack the batch
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Calculate validation metrics
        # labels = FILL
        predicted_labels = torch.argmax(eval_output.logits, dim=1)
        # print(f"labels {labels} predicted_labels {predicted_labels}")

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(predicted_labels, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:  50%|█████     | 1/2 [00:05<00:05,  5.83s/it]


	 - Train loss: 1.0398
	 - Validation Accuracy: 0.8474
	 - Validation Precision: 0.4209
	 - Validation Recall: 0.6076
	 - Validation AUROC: 0.7019



Epoch: 100%|██████████| 2/2 [00:11<00:00,  5.91s/it]


	 - Train loss: 0.2865
	 - Validation Accuracy: 0.9089
	 - Validation Precision: 0.5962
	 - Validation Recall: 0.5679
	 - Validation AUROC: 0.7178



### Test on a specific sentence, see the outcome

In [ ]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here

Fine tuning the entire model consistently yields better performance across all metrics and results in a lower training loss, although it requires more training time due to the larger number of trainable parameters.
In contrast, fine tuning only the classifier treats GPT-2 as a fixed feature encoder with general purpose, which limits the classifier’s ability to adapt to the patterns of our dataset and therefore leads to weaker overall performance.


| Model Setting  | Accuracy | Precision | Recall | AUROC | Train Time | Train Loss |
| -------------- | -------- | --------- | ------ | ----- | ---------- |------|
| Fine tune classifir only   | 0.9089     | 0.5962      | 0.5679   | 0.7178  | 5.83s/it      | 0.0006|
| Fine tune the entire model | 0.9866   | 0.8900      | 0.8448 | 0.878 | 25.26s/it     |0.2865|





Fine tune the entire model:

Epoch:  50%|█████     | 1/2 [00:25<00:25, 25.24s/it]
	 - Train loss: 0.0017
	 - Validation Accuracy: 0.9848
	 - Validation Precision: 0.8833
	 - Validation Recall: 0.8293
	 - Validation AUROC: 0.8711

Epoch: 100%|██████████| 2/2 [00:50<00:00, 25.26s/it]
	 - Train loss: 0.0006
	 - Validation Accuracy: 0.9866
	 - Validation Precision: 0.8900
	 - Validation Recall: 0.8448
	 - Validation AUROC: 0.878

Fine tune classifier:

Epoch:  50%|█████     | 1/2 [00:05<00:05,  5.83s/it]
	 - Train loss: 1.0398
	 - Validation Accuracy: 0.8474
	 - Validation Precision: 0.4209
	 - Validation Recall: 0.6076
	 - Validation AUROC: 0.7019

Epoch: 100%|██████████| 2/2 [00:11<00:00,  5.91s/it]
	 - Train loss: 0.2865
	 - Validation Accuracy: 0.9089
	 - Validation Precision: 0.5962
	 - Validation Recall: 0.5679
	 - Validation AUROC: 0.7178

